# Desafio 1
***
<ul><li>Enunciado:</li></ul>
Este desafio consiste na elaboração de um script que realize as seguintes tarefas:

In [1]:
# Importando bibliotecas que serão utilizadas no desafio
import pandas as pd
import numpy as np
import datetime as dt


<br><h3><ul><li>
 Abra o arquivo Mediçao.xls e transforme-o em DataFrame com a Biblioteca Pandas,este DataFrame deverá ter as seguintes colunas:
</li></ul></h3>
['RecNo', 'MeaValue', 'Weight', 'Time', 'Date', 'DataGroupIndex']

In [2]:
df = pd.read_excel('Medicao.xls')
df.head(3)


,RecNo,MeaValue,Weight,Time,Date,DataGroupIndex
0,1013,"60,8",A,6:39:12,2020-12-7,1
1,1014,"60,5",A,6:39:13,2020-12-7,1
2,1015,"43,1",A,6:39:14,2020-12-7,1


<br><h3><ul><li>
Selecione somente os intervalos contínuos de 1 minuto da coluna Time. 
</li></ul></h3>
Exemplo: o script   deve   selecionar   somente   os intervalos   contínuos   que   se   iniciem   em HH:mm:00 e terminem em HH:mm:59, onde HH (hora) mm (minuto) devem ser os mesmos no intervalo;
<br><br>Dica: descarte os dados não contínuos do DataFrame;
<br><h3><ul><li>
Em cada intervalo selecionado, faça a média simples dos dados da coluna MeaValue.
</li></ul></h3>


In [3]:
# Realizando análise exploratória das estruturas de dados do DataFrame
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2268 entries, 0 to 2267
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   RecNo           2268 non-null   int64 
 1   MeaValue        2268 non-null   object
 2   Weight          2268 non-null   object
 3   Time            2268 non-null   object
 4   Date            2268 non-null   object
 5   DataGroupIndex  2268 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 106.4+ KB


<br><h3><ul><li>
Crie um DataFrame com as seguintes colunas:
</li></ul></h3>
<h6><ol><li>
ValorMedio: com o valor médio de cada intervalo;
</li><li>
Hora: com a hora correspondente no formato HH:mm;
</li><li>
Data: com o dia de medição (presente na coluna Date);
</li></ol></h6>

In [4]:
# Tratando os tipos de data das columas 'Time' e 'Date'
df['Time'] =  pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time
df['Date'] =  pd.to_datetime(df['Date'], format='%Y-%m-%d')

# Substituindo ',' por '.' e tratatando os tipos de dados da coluna 'MeaValue'
df['MeaValue'] = df['MeaValue'].str.replace(',','.')
df['MeaValue'] = df['MeaValue'].apply(np.float64)

# Imprimindo mudança da substituição na coluna 'MeaValue'
df.head(3)


,RecNo,MeaValue,Weight,Time,Date,DataGroupIndex
0,1013,60.8,A,06:39:12,2020-12-07,1
1,1014,60.5,A,06:39:13,2020-12-07,1
2,1015,43.1,A,06:39:14,2020-12-07,1


In [5]:
# Imprimindo as mudanças feitas nas estruturas de dados das colunas alteradas
df.dtypes


RecNo                      int64
MeaValue                 float64
Weight                    object
Time                      object
Date              datetime64[ns]
DataGroupIndex             int64
dtype: object

In [6]:
# Criando uma cópia para podermos trabalhar e realizar alterações no DataFrame e termos um backup
df_copia = df


In [7]:
# Selecione somente os intervalos contínuos de 1 minuto da coluna Time
df_copia['Time'] = df_copia['Time'].apply(lambda t: t.strftime('%H:%M'))

# Imprimindo mudança
df_copia.head(3)


,RecNo,MeaValue,Weight,Time,Date,DataGroupIndex
0,1013,60.8,A,06:39,2020-12-07,1
1,1014,60.5,A,06:39,2020-12-07,1
2,1015,43.1,A,06:39,2020-12-07,1


In [10]:
# Criando um DataFrame que irá armazenar nossos dados e resultados finais
df_final = pd.DataFrame(columns=['ValorMedio'])

# Iremos atribuir a este DataFrame a média arredondada em uma casa decimal de forma agrupada pelas colunas 'Time' e 'Date'
df_final['ValorMedio'] = df_copia.groupby(['Time', 'Date'])[['MeaValue']].mean().round(1)
df_final.head(2)

# Normalizando a indexação do DataFrame final
df_final.reset_index(inplace=True)

# Alterando os nomes das colunas
df_final = df_final.rename(columns = {'Time':'Hora', 'Date':'Data'})

# Imprimindo resultados das mudanças
df_final.head(3)


,Hora,Data,ValorMedio
0,06:39,2020-12-07,46.8
1,06:40,2020-12-07,42.8
2,06:41,2020-12-07,38.2


In [9]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Hora        46 non-null     object        
 1   Data        46 non-null     datetime64[ns]
 2   ValorMedio  46 non-null     float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 1.2+ KB


In [10]:
# Criando um DataFrame para fazer a contagem de segundos contínuos
df_sec_continuous = pd.DataFrame(columns=['Contagem'])

# Fazendo a contagem de continuidade de segundos
df_sec_continuous = df_copia.groupby(['Time', 'Date'])[['MeaValue']].count()

# Filtrando minutos sem continuidade
df_sec_continuous = df_sec_continuous[df_sec_continuous.MeaValue == 60]

# Normalizando a indexação do DataFrame
df_sec_continuous.reset_index(inplace=True)

# Alterando os nomes das colunas
df_sec_continuous = df_sec_continuous.rename(columns = {'Time':'Hora', 'Date':'Data'})

# Atualizando dataframe final com apenas os minutos com segundos continuoes entre 00 e 59 segundos
df_final = df_final.merge(df_sec_continuous, how='left').dropna()

# Fazendo o drop da coluna de contagem de continuidade de segundos
df_final = df_final.drop(['MeaValue'], axis=1)

# Imprimindo resultados
display(df_final)


,Hora,Data,ValorMedio
2,06:41,2020-12-07,38.2
3,06:42,2020-12-07,38.0
4,06:43,2020-12-07,37.1
5,06:44,2020-12-07,37.6
9,06:50,2020-12-07,38.3
10,06:51,2020-12-07,38.6
11,06:52,2020-12-07,39.4
12,06:53,2020-12-07,40.4
13,06:54,2020-12-07,39.9
16,06:57,2020-12-07,41.2


***
<h3><ul><li>
O Desafio deve ser feito em duplas, trios ou individual. 
</li><li>
O DataFrame deve ser salvo em .xls com o nome de “Desafio1 ;seus nomes;.xls”
</li><li>
Enviar por slack junto com o script em .ipynb. Todos os passos feitos no script devem estar documentados usando Markdown.
</li></ul></h3>

In [12]:
# Salvando data frame final
df_final.to_excel('Desafio_1-Daniel-Harao-Rhayza.xls', sheet_name='Sheet_name_1')


C:\Users\017924631\AppData\Local\Temp\ipykernel_2668\723124845.py:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df_final.to_excel('Desafio_1-Daniel-Harao-Rhayza.xls', sheet_name='Sheet_name_1')
